# PySpark DataFrame Manipulation

### Imports

In [1]:
# Starting the PySpark session
from pyspark import SparkContext
from pyspark.sql import SparkSession
# Encoding categorical data
import pyspark.sql.functions as F
from pyspark.sql import functions as f
# PySpark Data Types
from pyspark.sql.types import *
# Add DataFrame columns from Python lists,
# and other PySpark Aggregate Functions
from pyspark.sql.functions import (
    array,
    col,
    mean,
    min,
    max,
)
from pyspark.sql import functions
from pyspark.sql import types as t
# Creating pyspark literals
from pyspark.sql.functions import lit
# Creating PySpark Columns
from pyspark.sql.column import Column
# Numpy Array functions
import numpy as np
# Helper functions
import util

### Start the PySpark Session

In [2]:
sc = SparkContext()

In [3]:
spark = SparkSession \
    .builder \
    .appName("Python Spark regression example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

### Lets make Pyspark Dataframe from titanic csv file 

In [4]:
df = spark.read.csv('titanic.csv',header=True, inferSchema = True)

### Look at the First 5 Rows

In [5]:
df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

### Describe the Dataset

In [6]:
df.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

*From the original assignment in DS 1.1:*

VARIABLE DESCRIPTIONS:
survival        Survival
                (0 = No; 1 = Yes)
pclass          Passenger Class
                (1 = 1st; 2 = 2nd; 3 = 3rd)
name            Name
sex             Sex
age             Age
sibsp           Number of Siblings/Spouses Aboard
parch           Number of Parents/Children Aboard
ticket          Ticket Number
fare            Passenger Fare
cabin           Cabin
embarked        Port of Embarkation
                (C = Cherbourg; Q = Queenstown; S = Southampton)

In [7]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



### How many of Age values are empty (or null)?

For this question, I used values provided by table in the `df.describe.show()` cell above.

In [8]:
num_rows_overall = 891
num_values_in_age = 714
num_null_in_age = num_rows_overall - num_values_in_age
print(f'The number of null values in the Age column: {num_null_in_age}.')

The number of null values in the Age column: 177.


### Create a new column as gender, when Sex is female it is zero when sex is male it is one

Background info on how the next two cells work can be found on [this blog post](https://towardsdatascience.com/5-ways-to-add-a-new-column-in-a-pyspark-dataframe-4e75c2fd8c08), credit to the author Rahul Agarwal for how User-Defined Functions work in PySpark.

In [9]:
def determine_binary_gender(sex):
    '''Return 0 for female and 1 for male.'''
    if sex == 'male':
        return 1
    # all non-males will be 0
    return 0
        
# convert determine_binary_gender to a UDF Function 
udf_determine_binary_gender = F.udf(determine_binary_gender, IntegerType())

In [10]:
# add the new column to the DataFrame
df = df.withColumn("Gender", udf_determine_binary_gender("Sex"))
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Gender|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|     1|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|     0|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|     0|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|     0|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|     1|
|          6|       0|  

### Check the Shape of the DataFrame

In [11]:
print((df.count(), len(df.columns)))

(891, 13)


### List all of the Ages that are not null

In [12]:
# grab ages column, by getting underlying RDD
ages = df.rdd.map(lambda r: r.Age).collect()

In [13]:
# print all integer values
for age in ages:
    if age is not None:
        print(age)

22.0
38.0
26.0
35.0
35.0
54.0
2.0
27.0
14.0
4.0
58.0
20.0
39.0
14.0
55.0
2.0
31.0
35.0
34.0
15.0
28.0
8.0
38.0
19.0
40.0
66.0
28.0
42.0
21.0
18.0
14.0
40.0
27.0
3.0
19.0
18.0
7.0
21.0
49.0
29.0
65.0
21.0
28.5
5.0
11.0
22.0
38.0
45.0
4.0
29.0
19.0
17.0
26.0
32.0
16.0
21.0
26.0
32.0
25.0
0.83
30.0
22.0
29.0
28.0
17.0
33.0
16.0
23.0
24.0
29.0
20.0
46.0
26.0
59.0
71.0
23.0
34.0
34.0
28.0
21.0
33.0
37.0
28.0
21.0
38.0
47.0
14.5
22.0
20.0
17.0
21.0
70.5
29.0
24.0
2.0
21.0
32.5
32.5
54.0
12.0
24.0
45.0
33.0
20.0
47.0
29.0
25.0
23.0
19.0
37.0
16.0
24.0
22.0
24.0
19.0
18.0
19.0
27.0
9.0
36.5
42.0
51.0
22.0
55.5
40.5
51.0
16.0
30.0
44.0
40.0
26.0
17.0
1.0
9.0
45.0
28.0
61.0
4.0
1.0
21.0
56.0
18.0
50.0
30.0
36.0
9.0
1.0
4.0
45.0
40.0
36.0
32.0
19.0
19.0
3.0
44.0
58.0
42.0
24.0
28.0
34.0
45.5
18.0
2.0
32.0
26.0
16.0
40.0
24.0
35.0
22.0
30.0
31.0
27.0
42.0
32.0
30.0
16.0
27.0
51.0
38.0
22.0
19.0
20.5
18.0
35.0
29.0
59.0
5.0
24.0
44.0
8.0
19.0
33.0
29.0
22.0
30.0
44.0
25.0
24.0
37.0
54.0
29.0
62.0
3

### Slice the dataframe for those whose Embarked section was 'C'

In [14]:
df.filter(df.Embarked == 'C').show()

+-----------+--------+------+--------------------+------+----+-----+-----+-------------+--------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|       Ticket|    Fare|Cabin|Embarked|Gender|
+-----------+--------+------+--------------------+------+----+-----+-----+-------------+--------+-----+--------+------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|     PC 17599| 71.2833|  C85|       C|     0|
|         10|       1|     2|Nasser, Mrs. Nich...|female|14.0|    1|    0|       237736| 30.0708| null|       C|     0|
|         20|       1|     3|Masselmani, Mrs. ...|female|null|    0|    0|         2649|   7.225| null|       C|     0|
|         27|       0|     3|Emir, Mr. Farred ...|  male|null|    0|    0|         2631|   7.225| null|       C|     1|
|         31|       0|     1|Uruchurtu, Don. M...|  male|40.0|    0|    0|     PC 17601| 27.7208| null|       C|     1|
|         32|       1|     1|Spencer, Mr

### Describe a specific column 

I used 'Embarked'

In [15]:
df.select('Embarked').describe().show()

+-------+--------+
|summary|Embarked|
+-------+--------+
|  count|     889|
|   mean|    null|
| stddev|    null|
|    min|       C|
|    max|       S|
+-------+--------+



### How many unique values does the 'Embarked' have?

In [16]:
# get the unique values from the column
unique = df.select('Embarked').distinct()
# unwrap the Row objects
unique_values = unique.rdd.map(lambda r: r.Embarked).collect()
# count the number of values, NULL should be excluded
nunique = len([val for val in unique_values if val is not None])
print(f"There are {nunique} unique values in the 'Embarked' column.")

There are 3 unique values in the 'Embarked' column.


### Show the Counts for Each Unique Value in 'Embarked'

AKA, do the PySpark version of Pandas' `value_counts()` function.

In [17]:
# dropping NaN values
df = df.dropna()
# showing the frequency distribution for the Embarked column
df.groupBy('Embarked').count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|    2|
|       C|   65|
|       S|  116|
+--------+-----+



### Export as JSON

In [18]:
value_counts = df.groupBy('Embarked').count()
value_counts.write.json('Embarked_Col')

### What is the average Age for female and male (based on sex) for those who have 'Embarked' on section 'C'?

In [19]:
def average_age(sex):
    '''Returns the average age of men or women, whom Embarked from section C.'''
    ages = util.get_ages(df, sex)
    # return the mean of the ages
    return np.mean(ages)

In [20]:
# average age for men
men_avg = round(average_age('male'), 1)
print(f"The average age for men who embarked from 'C' section: {men_avg} years.")

The average age for men who embarked from 'C' section: 39.8 years.


In [21]:
# average age for women
women_avg = round(average_age('female'), 1)
print(f"The average age for women who embarked from 'C' section: {women_avg} years.")

The average age for women who embarked from 'C' section: 35.4 years.


### Which Age is the oldest for female and male (based on sex) for those who have 'Embarked' on section 'C'?

In [26]:
def get_max_age(sex):
    '''Return the max age for men or women '''
    ages = util.get_ages(df, sex)
    # return the max value
    return np.max(ages)

# display the max value for men
max_men = get_max_age('male')
print(
        "The oldest age amongst the men " +
        "who embarked from the 'C' section: " +
        f"{max_men} years."
)

The oldest age amongst the men who embarked from the 'C' section: 71.0 years.


In [27]:
# display the max value for women
max_women = get_max_age('female')
print(
        "The oldest age amongst the women " +
        "who embarked from the 'C' section: " +
        f"{max_women} years."
)

The oldest age amongst the women who embarked from the 'C' section: 60.0 years.


### What are the number of female and male at each Passenger Class?

In [28]:
def numbers_of_male_female(passenger_class, sex):
    '''Return the numbers of male and female, for a given passenger class.'''
    num_people = df.filter((df.Pclass == passenger_class) & (df.Sex == sex)).rdd.map(lambda person: person.Age).collect()
    return len(num_people)

# get a list of all passenger classes, as a list
unique_classes = df.select('Pclass').distinct().rdd.map(lambda person: person.Pclass).collect()
# iterate over the classes
for p_class in unique_classes:
    # get the numbers of men and women
    num_male = numbers_of_male_female(p_class, 'male')
    num_female = numbers_of_male_female(p_class, 'female')
    # print both
    print(f"There were {num_male} men and {num_female} women in Passenger class {p_class}.")

There were 84 men and 74 women in Passenger class 1.
There were 5 men and 5 women in Passenger class 3.
There were 6 men and 9 women in Passenger class 2.


### What percent of passengers embarked at C?

In [29]:
num_c_passengers = len(
    df.filter(df.Embarked == 'C').rdd.collect()
)

percentage = round(num_c_passengers / df.count(), 3) * 100
print(f"{percentage}% for passengers embarked at C.")

35.5% for passengers embarked at C.


### What percent of female passengers embarked at C?

In [30]:
# the function gets the ages of female passengers who embarked from C,
# and then we also take the length
num_females_at_c = len(util.get_ages(df, 'female')) 
# get the total number of female passengers
num_females = len(df.filter(df.Sex == 'female').rdd.collect())
# compute and display the percentage
percentage = round(num_females_at_c / num_females , 3) * 100
print(f"{percentage}% for female passengers embarked at C.")

38.6% for female passengers embarked at C.
